## 예제2.1 토큰화 코드 

In [1]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


## 예제 2.2 토큰 아이디에서 벡터로 변환

In [2]:
import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0) # (1, 5, 16)
input_embeddings.shape

torch.Size([1, 5, 16])

In [3]:
input_embeddings

tensor([[[ 5.0336e-02,  2.9919e-01,  4.8854e-01, -1.2208e+00, -1.5991e-01,
           9.5425e-01,  1.5653e+00, -1.7314e+00,  6.7242e-01,  2.2700e-03,
          -1.2909e+00,  1.5111e+00,  7.8708e-01, -5.3714e-01,  8.9638e-01,
          -2.8776e-01],
         [ 1.8069e-01, -1.3104e+00, -2.5331e+00,  1.0643e+00, -7.6283e-01,
           3.9897e-01,  3.4030e-01, -2.4216e-01, -6.9131e-01, -2.5506e-01,
           6.1183e-01, -9.9394e-01, -2.4111e-01,  2.8682e-01, -2.2687e-02,
           1.0095e+00],
         [ 1.0018e+00,  9.4114e-01,  2.7150e-01, -1.5196e-01, -1.6213e-01,
           1.8214e-01,  8.1594e-01, -7.0765e-02,  9.5699e-01, -1.5790e+00,
           1.1889e+00, -4.1294e-01,  9.2490e-01,  2.8000e-01,  1.1623e+00,
          -6.7500e-01],
         [-1.0234e+00, -1.6055e+00, -1.6431e-01,  2.6785e-02,  5.7909e-01,
           6.7505e-01,  2.1682e+00,  7.2853e-01,  4.1666e-01, -6.8641e-01,
          -2.1802e-01, -5.8620e-02,  2.6941e-01,  4.0510e-01, -4.6775e-01,
          -1.0920e+00],
    

## 예제 2.3 절대적 위치 인코딩

In [4]:
embedding_dim = 16
max_position = 12
# 토큰 임베딩 층 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
# 위치 인코딩 층 생성
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
token_embeddings = token_embeddings.unsqueeze(0) # (1, 5, 16)
# 토큰 임베딩과 위치 인코딩을 더해 최종 입력 임베딩 생성
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

In [5]:
input_embeddings[0]

tensor([[ 1.5433,  1.2280, -1.0297,  1.5270, -1.5361, -0.5542,  0.7781, -0.4501,
          0.7248, -1.0035,  1.4972, -0.0194,  0.5586,  0.2413, -0.7455, -0.4547],
        [ 0.7163, -0.5858,  0.3381, -0.6159, -0.9343,  1.5423, -0.5425, -0.8263,
          1.5400, -2.6927, -1.6165, -1.4989,  1.0501,  1.2490,  0.1997,  1.2883],
        [-1.8502,  2.7743, -1.3057,  1.0715,  1.0108,  1.5909,  1.0176,  1.0362,
         -3.5778, -0.7900,  1.5996, -0.9310,  0.3907, -1.2328, -0.5081, -0.5500],
        [-0.5528, -1.4244, -0.5564, -0.7580, -0.1572,  0.2040, -2.0029,  2.1319,
          1.3805,  1.6805, -1.2635,  0.1477, -0.3059,  1.3573,  1.4138,  0.6202],
        [ 1.4873,  2.7248, -0.5334,  1.8302, -0.0715,  0.2174, -0.9959, -0.5355,
          0.1605,  1.5293,  1.6743,  1.4735,  0.4054,  0.7705, -0.0241, -1.9884]],
       grad_fn=<SelectBackward0>)

## 예제 2.4 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

In [6]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)
# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 16)
keys = weight_k(input_embeddings) # (1, 5, 16)
values = weight_v(input_embeddings) # (1, 5, 16)

In [7]:
weight_q

Linear(in_features=16, out_features=16, bias=True)

In [8]:
input_embeddings.shape

torch.Size([1, 5, 16])

In [9]:
querys.shape

torch.Size([1, 5, 16])

## 예제 2.5. 스케일 점곱 방식의 어텐션

In [10]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
	# 쿼리의 마지막 차원 크기를 가져옴 (head_dim)
	dim_k = querys.size(-1) # 16
	# 쿼리와 키의 내적을 계산하고 스케일링 수행
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
	# 소프트맥스를 통해 어텐션 가중치 계산
	weights = F.softmax(scores, dim=-1)
	# 가중치와 값을 곱해 최종 어텐션 결과 반환
	return weights @ values

## 예제 2.6. 어텐션 연산의 입력과 출력

In [11]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)
# 원본 입력 형태:  torch.Size([1, 5, 16])
# 어텐션 적용 후 형태:  torch.Size([1, 5, 16])

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


## 예제 2.7. 어텐션 연산을 수행하는 AttentionHead 클래스

In [12]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

## 예제 2.8. 멀티 헤드 어텐션 구현

In [13]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    # 헤드의 개수와 인과성 마스킹 여부를 저장
    self.n_head = n_head
    self.is_causal = is_causal
    # 쿼리, 키, 값 벡터를 생성하기 위한 선형 층
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    # 멀티 헤드의 출력을 합치고 변환하기 위한 선형 층
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    # 입력 텐서의 크기를 가져옴 (배치 크기, 시퀀스 길이, 임베딩 차원)
    B, T, C = querys.size()
    # 쿼리, 키, 값 벡터를 각각의 헤드로 분할하고 차원을 재배열
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    # 어텐션 연산 수행
    attention = compute_attention(querys, keys, values, self.is_causal)
    # 어텐션 결과를 원래 차원으로 되돌리고 연결
    output = attention.transpose(1, 2).contiguous().view(B, T, C)
    # 최종 선형 변환 적용
    output = self.concat_linear(output)
    return output

# 4개의 어텐션 헤드 사용
n_head = 4
# 멀티헤드 어텐션 객체 생성
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
# 입력에 대해 멀티헤드 어텐션 연산 수행
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

## 예제 2.9. 층 정규화 코드

In [14]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape # torch.Size([1, 5, 16])

torch.Size([1, 5, 16])

In [15]:
norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data

# (tensor([[ 2.2352e-08, -1.1176e-08, -7.4506e-09, -3.9116e-08, -1.8626e-08]]),
#  tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

(tensor([[-7.4506e-09,  7.4506e-09,  1.1176e-08,  1.4901e-08,  0.0000e+00]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

## 예제 2.10. 피드 포워드 층 코드

In [16]:
class PreLayerNormFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
    self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃 층 1
    self.dropout2 = nn.Dropout(dropout) # 드랍아웃 층 2
    self.activation = nn.GELU() # 활성 함수
    self.norm = nn.LayerNorm(d_model) # 층 정규화

  def forward(self, src):
    x = self.norm(src)
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x

## 예제 2.11. 인코더 층

In [17]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead) # 트랜스포머 인코더의 멀티헤드 어텐션 계층
    self.norm1 = nn.LayerNorm(d_model) # 입력값을 정규화하는 층 정규화 계층
    self.dropout1 = nn.Dropout(dropout) # 과적합 방지를 위한 드롭아웃 계층
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드포워드 신경망 계층

  def forward(self, src):
    norm_x = self.norm1(src) # 입력값 정규화
    attn_output = self.attn(norm_x, norm_x, norm_x) # 셀프 어텐션 수행
    x = src + self.dropout1(attn_output) # 잔차 연결로 어텐션 결과를 원본 입력과 더함

    # 피드포워드 신경망을 통과
    x = self.feed_forward(x) # 피드포워드 계층에서 특징 추출
    return x

## 예제 2.12. 인코더 구현

In [18]:
import copy

# 모듈의 복사본을 N개 생성하는 함수
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

# 트랜스포머 인코더 클래스 정의
class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    # 인코더 층을 num_layers만큼 복제
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers # 인코더 층의 개수
    self.norm = norm # 층 정규화

  def forward(self, src):
    output = src
    # 각 인코더 층을 순차적으로 통과
    for mod in self.layers:
        output = mod(output)
    return output

## 예제 2.13. 디코더에서 어텐션 연산(마스크 어텐션)

In [19]:
def compute_attention(querys, keys, values, is_causal=False):
    # 어텐션 스코어를 계산하기 위한 차원 크기 추출
    dim_k = querys.size(-1) # 16
    
    # 어텐션 스코어 계산: Q * K^T / sqrt(d_k)
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1, 5, 5)
    
    # 인과적 마스킹(Causal Masking) 적용
    if is_causal:
        # 쿼리와 키의 시퀀스 길이 추출
        query_length = querys.size(2)
        key_length = keys.size(2)
        
        # 하삼각 행렬 마스크 생성
        temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)
        
        # 마스크를 적용하여 미래 토큰에 대한 어텐션을 차단
        scores = scores.masked_fill(temp_mask == False, float("-inf"))
    
    # 소프트맥스를 통한 어텐션 가중치 계산
    weights = F.softmax(scores, dim=-1) # (1, 5, 5)
    
    # 가중치와 값을 곱하여 최종 어텐션 결과 반환
    return weights @ values # (1, 5, 16)

## 예제 2.14. 크로스 어텐션이 포함된 디코더 층

In [21]:
class TransformerDecoderLayer(nn.Module):
  """트랜스포머 디코더 층 클래스
  
  Args:
      d_model: 모델의 차원 크기
      nhead: 멀티헤드 어텐션의 헤드 수
      dim_feedforward: 피드포워드 신경망의 은닉층 차원
      dropout: 드롭아웃 비율
  """
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    super().__init__()
    # 셀프 어텐션 층
    self.self_attn = MultiheadAttention(d_model, d_model, nhead)
    # 크로스 어텐션 층 
    self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
    # 피드포워드 신경망
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    # 층 정규화
    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    # 드롭아웃
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)

  def forward(self, tgt, encoder_output, is_causal=True):
    """
    Args:
        tgt: 디코더 입력 텐서
        encoder_output: 인코더의 출력 텐서
        is_causal: 인과적 마스킹 사용 여부
    """
    # 셀프 어텐션 연산
    x = self.norm1(tgt)
    x = x + self.dropout1(self.self_attn(x, x, x, is_causal=is_causal))
    # 크로스 어텐션 연산
    x = self.norm2(x)
    x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))
    # 피드 포워드 연산
    x = self.feed_forward(x)
    return x

## 예제 2.15. 디코더 구현

In [22]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerDecoder(nn.Module):
  def __init__(self, decoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(decoder_layer, num_layers)
    self.num_layers = num_layers

  def forward(self, tgt, src):
    output = tgt
    for mod in self.layers:
        output = mod(tgt, src)
    return output